In [4]:
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import sys
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

FEATURE_LIST = ['X_power_1_2','X_coh_1_2','X_gc_1_2']
FEATURE_VECTOR = FEATURE_LIST

UMC_PATH = "/hpc/home/mk423/Anxiety/Universal-Mouse-Code/"
DATA_PATH = "/work/mk423/Anxiety/Anxiety_Network_Generation_Data.pkl"
MODEL_PATH = "../Models/"
MT_MODEL_PATH = MODEL_PATH + "Positive_MT_10_res_loss_10_power_features.pt"
sys.path.append(UMC_PATH)
sys.path.append(DATA_PATH)

#from dCSFA_model import dCSFA_model
import umc_data_tools as umc_dt
from dCSFA_NMF import dCSFA_NMF

if torch.cuda.is_available():
    device="cuda:0"
else:
    device="cpu"


print("Using device: %s"%(device))

#For Consistency
RANDOM_STATE=42

Using device: cuda:0


In [5]:
with open(DATA_PATH,"rb") as f:
    dataDict = pickle.load(f)

model = torch.load(MT_MODEL_PATH,map_location='cpu')
model.device = "cpu"
model.eval()

dCSFA_NMF(
  (recon_loss_f): MSELoss()
  (Encoder): Sequential(
    (0): Linear(in_features=5152, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear(in_features=256, out_features=20, bias=True)
    (4): Softplus(beta=1, threshold=20)
  )
)

In [7]:
X_train = dataDict['X_train']
y_train = dataDict['y_train']
y_mouse_train = dataDict['y_mouse_train']
y_exp_train = dataDict['y_exp_train']

X_val = dataDict['X_val']
y_val = dataDict['y_val']
y_mouse_val = dataDict['y_mouse_val']
y_exp_val = dataDict['y_exp_val']

X_test = dataDict['X_test']
y_test = dataDict['y_test']
y_mouse_test = dataDict['y_mouse_test']
y_exp_test = dataDict['y_exp_test']

In [9]:
y_pred, s = model.transform(X_test,None)[2:]

In [16]:
results_dict = {
            "epm-mean":[],
            "flx-mean":[],
            "oft-mean":[],
    
            "epm-sterr":[],
            "flx-sterr":[],
            "oft-sterr":[],
    
            "network":[],
}
for i in range(20):
    for exp in ['epm','flx','oft']:
        
        mw_mean, mw_std = umc_dt.get_mean_std_err_auc(y_pred[y_exp_test==exp],y_test[y_exp_test==exp],
                                                      y_mouse_test[y_exp_test==exp],s[y_exp_test==exp,i].reshape(-1,1),True)
        
        results_dict[exp + "-mean"].append(mw_mean)
        results_dict[exp + "-sterr"].append(mw_std)
    
    results_dict["network"].append(i)

Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative samples  0
Mouse  Mouse99021  has only one class - AUC cannot be calculated
n_positive samples  2025
n_negative samples  0
Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative samples  0
Mouse  Mouse99021  has only one class - AUC cannot be calculated
n_positive samples  2025
n_negative samples  0
Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative samples  0
Mouse  Mouse99021  has only one class - AUC cannot be calculated
n_positive samples  2025
n_negative samples  0
Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative samples  0
Mouse  Mouse99021  has only one class - AUC cannot be calculated
n_positive samples  2025
n_negative samples  0
Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative sam

/hpc/home/mk423/Anxiety/Universal-Mouse-Code/umc_data_tools.py:274: RuntimeWarning: invalid value encountered in double_scalars
  std = np.std(auc_list) / np.sqrt(len(auc_list)-1)


Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative samples  0
Mouse  Mouse99021  has only one class - AUC cannot be calculated
n_positive samples  2025
n_negative samples  0
Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative samples  0
Mouse  Mouse99021  has only one class - AUC cannot be calculated
n_positive samples  2025
n_negative samples  0
Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative samples  0
Mouse  Mouse99021  has only one class - AUC cannot be calculated
n_positive samples  2025
n_negative samples  0
Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative samples  0
Mouse  Mouse99021  has only one class - AUC cannot be calculated
n_positive samples  2025
n_negative samples  0
Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative sam

In [18]:
df = pd.DataFrame.from_dict(results_dict)
df.to_csv("../Validations/all_network_aucs.csv")